In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
##Import library modul
# Pre-processing
import skimage.io as io
from skimage.transform import  rescale,resize
from skimage.util import img_as_uint,img_as_ubyte
from skimage.color import rgb2gray
from skimage import exposure
import os
import numpy as np
import cv2
from math import log10, sqrt
from wand.image import Image

# Extract features
import scipy.io as sio
from scipy.stats import skew
from scipy.stats import kurtosis
from skimage.feature import greycomatrix
import pandas as pd
import matplotlib.pyplot as plt
from skimage.filters import unsharp_mask, threshold_local
from skimage.morphology import closing, opening, square
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.color import label2rgb
import matplotlib.patches as mpatches
from tqdm.notebook import tqdm

# Train model
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import KernelPCA
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import tensorflow as tf
from sklearn.metrics import roc_curve, auc

In [ ]:
# =============================================================================
# im2double
# =============================================================================
def im2double(img):
    """ convert image to double format """
    min_val = np.min(img.ravel())
    max_val = np.max(img.ravel())
    out = (img.astype('float') - min_val) / (max_val - min_val)
    return out

# =============================================================================
# compute_14_features
# =============================================================================
def features_14(region):
    """ Compute 14 features """
    temp_array=region.reshape(-1)
    all_pixels=temp_array[temp_array!=0]
#    Area
    Area = np.sum(all_pixels)
#    mean
    density = np.mean(all_pixels)
#   Std
    std_Density = np.std(all_pixels)
#   skewness
    Skewness = skew(all_pixels)
#   kurtosis
    Kurtosis = kurtosis(all_pixels)
#   Energy
    ENERGY =np.sum(np.square(all_pixels))
#   Entropy
    value,counts = np.unique(all_pixels, return_counts=True)
    p = counts / np.sum(counts)
    p =  p[p!=0]
    ENTROPY =-np.sum( p*np.log2(p));
#   Maximum
    MAX = np.max(all_pixels)
#   Mean Absolute Deviation
    sum_deviation= np.sum(np.abs(all_pixels-np.mean(all_pixels)))
    mean_absolute_deviation = sum_deviation/len(all_pixels)
#   Median
    MEDIAN = np.median(all_pixels)
#   Minimum
    MIN = np.min(all_pixels)
#   Range
    RANGE = np.max(all_pixels)-np.min(all_pixels)
#   Root Mean Square
    RMS = np.sqrt(np.mean(np.square(all_pixels))) 
#    Uniformity
    UNIFORMITY = np.sum(np.square(p))

    features = np.array([Area, density, std_Density,
        Skewness, Kurtosis,ENERGY, ENTROPY,
        MAX, mean_absolute_deviation, MEDIAN, MIN, RANGE, RMS, UNIFORMITY])
    return features

In [ ]:
## Inisialisasi directory
base_dir = '../input/vinbigdata-chest-xray-resized-png-256x256'
second_dir = '../input/set-targeted-vinbigdata-chest-xray-resized'
train_dir = f'{base_dir}/train/'
test_dir = f'{base_dir}/test/'
output_dir = '.'

In [ ]:
train_csv = pd.read_csv(f'{second_dir}/labeld_train.csv', delimiter=',')
df_train = pd.DataFrame(train_csv)
class_normal = df_train[df_train['class_id']==0]
class_abnormal = df_train[df_train['class_id']==1]

In [ ]:
print('ori')
features=np.empty([1,70])#feature pool
# output_file = ['ori, global, adapt, um']
for i in range(2):
    label_class = i # 0:Normal, 1: Abnormal
    if label_class ==  0:
        output_filename = 'normal'
        enumerate_class = class_normal
    else:
        output_filename = 'abnormal'
        enumerate_class = class_abnormal
        
    for idx, img_name in enumerate(enumerate_class['image_id']):
#         if(idx%1000 ==0): print(idx)
        # Pre-processing image
        img_ori = plt.imread(train_dir+img_name+'.png')
        im8bit=img_as_ubyte(img_ori)
        # Extracting feature
        imrescale_2=(im8bit-np.min(im8bit))/(np.max(im8bit)-np.min(im8bit))
        textures=features_14(imrescale_2) #texture features

        glcms = greycomatrix(im8bit, [1], [0, 90, 180, 270]) #GLCM in eight directions
        glcm_features_4 = np.concatenate((features_14(im2double(glcms[:, :, 0, 0])), 
                                      features_14(im2double(glcms[:, :, 0, 1])),
                                      features_14(im2double(glcms[:, :, 0, 2])),
                                      features_14(im2double(glcms[:, :, 0, 3]))), axis=0)

        textures=np.concatenate((textures,glcm_features_4), axis=0).reshape(1,70)
        features=np.concatenate((features,textures), axis=0)
    features=np.delete(features, 0, 0)
    features=np.concatenate((features,label_class*np.ones(len(features)).reshape(len(features),1)), axis=1)
    sio.savemat('ori'+output_filename+'.mat', {'features': features})
    features=[]
    features=np.empty([1,70])

In [ ]:
print('global')
features=np.empty([1,70])#feature pool
# output_file = ['ori, global, adapt, um']
for i in range(2):
    label_class = i # 0:Normal, 1: Abnormal
    if label_class ==  0:
        output_filename = 'normal'
        enumerate_class = class_normal
    else:
        output_filename = 'abnormal'
        enumerate_class = class_abnormal
        
    for idx, img_name in enumerate(enumerate_class['image_id']):
#         if(idx%1000 ==0): print(idx)
        # Pre-processing image
        img_ori = plt.imread(train_dir+img_name+'.png')
        imhist=exposure.equalize_hist(img_ori)#global hist
        im8bit=img_as_ubyte(img_ori)
#       
        # Extracting feature
        imrescale_2=(im8bit-np.min(im8bit))/(np.max(im8bit)-np.min(im8bit))
        textures=features_14(imrescale_2) #texture features

        glcms = greycomatrix(im8bit, [1], [0, 90, 180, 270]) #GLCM in eight directions
        glcm_features_4 = np.concatenate((features_14(im2double(glcms[:, :, 0, 0])), 
                                      features_14(im2double(glcms[:, :, 0, 1])),
                                      features_14(im2double(glcms[:, :, 0, 2])),
                                      features_14(im2double(glcms[:, :, 0, 3]))), axis=0)

        textures=np.concatenate((textures,glcm_features_4), axis=0).reshape(1,70)
        features=np.concatenate((features,textures), axis=0)
    features=np.delete(features, 0, 0)
    features=np.concatenate((features,label_class*np.ones(len(features)).reshape(len(features),1)), axis=1)
    sio.savemat('global'+output_filename+'.mat', {'features': features})
    features=[]
    features=np.empty([1,70])

In [ ]:
print('adapt')
features=np.empty([1,70])#feature pool
# output_file = ['ori, global, adapt, um']
for i in range(2):
    label_class = i # 0:Normal, 1: Abnormal
    if label_class ==  0:
        output_filename = 'normal'
        enumerate_class = class_normal
    else:
        output_filename = 'abnormal'
        enumerate_class = class_abnormal
        
    for idx, img_name in enumerate(enumerate_class['image_id']):
#         if(idx%1000 ==0): print(idx)
        # Pre-processing image
        img_ori = plt.imread(train_dir+img_name+'.png')
        imhist=exposure.equalize_adapthist(img_ori, clip_limit=0.3)#adapt hist
        im8bit=img_as_ubyte(img_ori)
#       
        # Extracting feature
        imrescale_2=(im8bit-np.min(im8bit))/(np.max(im8bit)-np.min(im8bit))
        textures=features_14(imrescale_2) #texture features

        glcms = greycomatrix(im8bit, [1], [0, 90, 180, 270]) #GLCM in eight directions
        glcm_features_4 = np.concatenate((features_14(im2double(glcms[:, :, 0, 0])), 
                                      features_14(im2double(glcms[:, :, 0, 1])),
                                      features_14(im2double(glcms[:, :, 0, 2])),
                                      features_14(im2double(glcms[:, :, 0, 3]))), axis=0)

        textures=np.concatenate((textures,glcm_features_4), axis=0).reshape(1,70)
        features=np.concatenate((features,textures), axis=0)
    features=np.delete(features, 0, 0)
    features=np.concatenate((features,label_class*np.ones(len(features)).reshape(len(features),1)), axis=1)
    sio.savemat('adapt'+output_filename+'.mat', {'features': features})
    features=[]
    features=np.empty([1,70])

In [ ]:
print('um')
features=np.empty([1,70])#feature pool
# output_file = ['ori, global, adapt, um']
for i in range(2):
    label_class = i # 0:Normal, 1: Abnormal
    if label_class ==  0:
        output_filename = 'normal'
        enumerate_class = class_normal
    else:
        output_filename = 'abnormal'
        enumerate_class = class_abnormal
        
    for idx, img_name in enumerate(enumerate_class['image_id']):
#         if(idx%1000 ==0): print(idx)
        # Pre-processing image
        img_ori = Image(filename=train_dir+img_name+'.png')
        img_ori.unsharp_mask(radius=8, sigma=4, amount=3, threshold=0.01)
        img_um = np.array(img_ori)[:,:,0]
        im8bit=img_as_ubyte(img_um)
#       
        # Extracting feature
        imrescale_2=(im8bit-np.min(im8bit))/(np.max(im8bit)-np.min(im8bit))
        textures=features_14(imrescale_2) #texture features

        glcms = greycomatrix(im8bit, [1], [0, 90, 180, 270]) #GLCM in eight directions
        glcm_features_4 = np.concatenate((features_14(im2double(glcms[:, :, 0, 0])), 
                                      features_14(im2double(glcms[:, :, 0, 1])),
                                      features_14(im2double(glcms[:, :, 0, 2])),
                                      features_14(im2double(glcms[:, :, 0, 3]))), axis=0)

        textures=np.concatenate((textures,glcm_features_4), axis=0).reshape(1,70)
        features=np.concatenate((features,textures), axis=0)
    features=np.delete(features, 0, 0)
    features=np.concatenate((features,label_class*np.ones(len(features)).reshape(len(features),1)), axis=1)
    sio.savemat('um'+output_filename+'.mat', {'features': features})
    features=[]
    features=np.empty([1,70])

In [ ]:
# file_mat_normal = ['orinormal, globalnormal, adaptnormal, umnormal']
# file_mat_abnormal = ['oriabnormal, globalabnormal, adaptabnormal, umabnormal']
normal_features=sio.loadmat(os.path.join(output_dir,'globalnormal.mat')) 
normal_features=normal_features['features'] 

abnormal_features=sio.loadmat(os.path.join(output_dir,'globalabnormal.mat')) 
abnormal_features=abnormal_features['features']

X=np.concatenate((normal_features[:,:-1],abnormal_features[:,:-1]), axis=0)#data
y=np.concatenate((normal_features[:,-1],abnormal_features[:,-1]), axis=0)#target

# =============================================================================
# normalization
# =============================================================================
min_max_scaler=MinMaxScaler()
X = min_max_scaler.fit_transform(X) 
# =============================================================================
# feature reduction (K-PCA)
# =============================================================================
transformer = KernelPCA(n_components=64, kernel='linear')
X = transformer.fit_transform(X)
# =============================================================================
# devide data into test,train, and validation sets
# =============================================================================
# y = to_categorical(y)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1)

# X_train, X_val, y_train, y_val = train_test_split(
#     X_train, y_train, test_size=0.25, random_state=1)

# Model
svm_classifier = SVC(kernel='rbf')
svm_classifier.fit(X_train, y_train)
y_pred = svm_classifier.predict(X_test)

hasil_conf = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(hasil_conf)
hasil_report = classification_report(y_test, y_pred)
print("Classification Report:",)
print (hasil_report)
hasil_score = accuracy_score(y_test,y_pred)
print("Accuracy:",hasil_score)

In [ ]:
import pywt
def madev(d, axis=None):
    """ Mean absolute deviation of a signal """
    return np.mean(np.absolute(d - np.mean(d, axis)), 0)

def wavelet_denoising(x, wavelet='haar', level=2):
    coeff = pywt.wavedec(x, wavelet, mode='symmetric')
    sigma = (1/0.6745) * madev(coeff[-level])
    uthresh = sigma * np.sqrt(2 * np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='soft') for i in coeff[1:])
    return pywt.waverec(coeff, wavelet, mode='symmetric')

fig = plt.figure(figsize=(30, 30))
i = 0
j = 0
for i in range(4):
    for idx, img_name in enumerate(class_normal['image_id'][:5]):
        # Pre-processing image
        img_ori = plt.imread(train_dir+img_name+'.png') 
        im8bit=img_as_ubyte(img_um)
        if j<4:
            im8bit=img_as_ubyte(img_ori)
            # Adds a subplot at the 1st position
            fig.add_subplot(5,4, j+1)
            plt.imshow(im8bit, label='original', cmap='gray')
        elif j<8:
            imhist=exposure.equalize_hist(img_ori)#global hist
            im8bit=img_as_ubyte(imhist)
            # Adds a subplot at the 1st position
            fig.add_subplot(5,4, j+1)
            plt.imshow(im8bit, label='global', cmap='gray')
        elif j<12:
            imhist=exposure.equalize_adapthist(img_ori, clip_limit=0.3)#adapt hist
            im8bit=img_as_ubyte(imhist)
            # Adds a subplot at the 1st position
            fig.add_subplot(5,4, j+1)
            plt.imshow(im8bit, label='adapt', cmap='gray')
        else:
            img_ori = Image(filename=train_dir+img_name+'.png')
            img_ori.unsharp_mask(radius=8, sigma=4, amount=3, threshold=0.01)
            img_um = np.array(img_ori)[:,:,0]
            im8bit=img_as_ubyte(img_um)
            # Adds a subplot at the 1st position
            fig.add_subplot(5,4, j+1)
            plt.imshow(im8bit, label='unhsaped_mask', cmap='gray')
        j+=1
plt.savefig('image_norm.png')

In [ ]:
import pywt
def madev(d, axis=None):
    """ Mean absolute deviation of a signal """
    return np.mean(np.absolute(d - np.mean(d, axis)), 0)

def wavelet_denoising(x, wavelet='haar', level=2):
    coeff = pywt.wavedec(x, wavelet, mode='symmetric')
    sigma = (1/0.6745) * madev(coeff[-level])
    uthresh = sigma * np.sqrt(2 * np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='soft') for i in coeff[1:])
    return pywt.waverec(coeff, wavelet, mode='symmetric')

fig = plt.figure(figsize=(30, 30))
i = 0
j = 0
for i in range(4):
    for idx, img_name in enumerate(class_normal['image_id'][:5]):
        # Pre-processing image
        img_ori = plt.imread(train_dir+img_name+'.png') 
        im8bit=img_as_ubyte(img_um)
        if j<4:
            im8bit=img_as_ubyte(img_ori)
            # Adds a subplot at the 1st position
            fig.add_subplot(5,4, j+1)
            hist = np.histogram(im8bit, bins=np.arange(0, 256))
            sig_filtered = wavelet_denoising(hist[0])
            # showing image
            plt.plot(hist[1][:-1], hist[0], lw=2, color='steelblue')
            plt.plot(sig_filtered, label= 'wavelet filter', color='red')
        elif j<8:
            imhist=exposure.equalize_hist(img_ori)#global hist
            im8bit=img_as_ubyte(imhist)
            # Adds a subplot at the 1st position
            fig.add_subplot(5,4, j+1)
            hist = np.histogram(im8bit, bins=np.arange(0, 256))
            sig_filtered = wavelet_denoising(hist[0])
            # showing image
            plt.plot(hist[1][:-1], hist[0], lw=2, color='steelblue')
            plt.plot(sig_filtered, label= 'wavelet filter', color='red')
        elif j<12:
            imhist=exposure.equalize_adapthist(img_ori, clip_limit=0.3)#adapt hist
            im8bit=img_as_ubyte(imhist)
            # Adds a subplot at the 1st position
            fig.add_subplot(5,4, j+1)
            hist = np.histogram(im8bit, bins=np.arange(0, 256))
            sig_filtered = wavelet_denoising(hist[0])
            # showing image
            plt.plot(hist[1][:-1], hist[0], lw=2, color='steelblue')
            plt.plot(sig_filtered, label= 'wavelet filter', color='red')
        else:
            img_ori = Image(filename=train_dir+img_name+'.png')
            img_ori.unsharp_mask(radius=8, sigma=4, amount=3, threshold=0.01)
            img_um = np.array(img_ori)[:,:,0]
            im8bit=img_as_ubyte(img_um)
            # Adds a subplot at the 1st position
            fig.add_subplot(5,4, j+1)
            hist = np.histogram(im8bit, bins=np.arange(0, 256))
            sig_filtered = wavelet_denoising(hist[0])
            # showing image
            plt.plot(hist[1][:-1], hist[0], lw=2, color='steelblue')
            plt.plot(sig_filtered, label= 'wavelet filter', color='red')
        j+=1
plt.savefig('hist_image_norm.png')

In [ ]:
for idx, img_name in enumerate(df_train['image_id'][54:55]):
        # Pre-processing image
        img_ori = plt.imread(train_dir+img_name+'.png')
        img_ori=exposure.equalize_adapthist(img_ori, clip_limit=0.2)
        th = [cv2.threshold(img, np.mean(img), 255, cv2.THRESH_BINARY_INV)[1] for img in tqdm(img_ori)]
        img_ori2 = np.array(th)[:,:,0]
        img_ori2 = closing(img_ori2, square(4))
        img_ori2 = opening(img_ori2, square(10))
        img_ori2 = closing(img_ori2, square(1))
        img_ori2 = img_ori2.astype(int)
        cleared = clear_border(img_ori2)
        
        plt.imshow(cleared,cmap='gray')
        
        # label image regions
        label_image = label(cleared)
        # to make the background transparent, pass the value of `bg_label`,
        # and leave `bg_color` as `None` and `kind` as `overlay`
        image_label_overlay = label2rgb(label_image, image=cleared, bg_label=0)

        fig, ax = plt.subplots(figsize=(10, 6))
        ax.imshow(img_ori, cmap='gray')

        for region in regionprops(label_image):
            # take regions with large enough areas
            if region.area >= 1000:
                # draw rectangle around segmented coins
                minr, minc, maxr, maxc = region.bbox
                rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                                          fill=False, edgecolor='red', linewidth=2)
                ax.add_patch(rect)

        ax.set_axis_off()
        plt.show()